<a href="https://colab.research.google.com/github/DaheeKo/Deep-learning-study/blob/main/CODES/Chap_09_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**9장 chap7 파이토치의 nn.Embedding()**

임베딩 층을 만들어 훈련데이터로부터 임베딩 벡터 학습 <- nn.Embedding() 사용해 구현


입력 시퀀스의 각 단어들 정수 인코딩 -> 임베딩 층의 입력으로 사용할 수 있음

=>  단어 -> 고유한 정수 인덱스 -> 임베딩 층 통과 -> 밀집벡터(임베딩 벡터)로 맵핑

* 임베딩 층 - 단어에게 부여된 정수를 인덱스로 하는 룩업 테이블. 
  
  단어의 정수 인덱스를 임베딩 층의 입력으로 사용 -> 룩업 테이블 -> 단어의 인덱스에 해당하는 행을 그 단어의 임베딩 벡터로 리턴 ->  임베딩 벡터가 모델의 입력이 되어 역전파 과정에서 학습

In [ ]:
import torch

# 임베딩 층 = 룩업테이블인 거 이해하기

train_data = 'you need to know how to code'
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑, 0과1은 비워줘야 하니까 i+2부터
vocab['<unk>'] = 0 
vocab['<pad>'] = 1
print(vocab)

# 단어 집합의 크기만큼의 행을 가지는 테이블 생성
embedding_table = torch.FloatTensor([[ 0.0,  0.0,  0.0],# unk
                               [ 0.0,  0.0,  0.0], # pad
                               [ 0.2,  0.9,  0.3], # to
                               [ 0.1,  0.5,  0.7], #you
                               [ 0.2,  0.1,  0.8], #know
                               [ 0.4,  0.1,  0.1], # need
                               [ 0.1,  0.8,  0.9], # how
                               [ 0.6,  0.1,  0.1]]) # code

sample = 'you need to run'.split() # 임의의 샘플
idxes=[]
# 각 단어 정수로 변환
for word in sample:
  try:
    idxes.append(vocab[word]) # 단어집합에 부여한 정수에서, word에 해당하는 값을 idxes에 넣음
  except KeyError: 
    idxes.append(vocab['<unk>']) # 단어 집합에 없는 단어일 경우 <unk>로 대체
idxes = torch.LongTensor(idxes)
print(idxes)

# 룩업 테이블
lookup_result = embedding_table[idxes,:] # 각 정수를 인덱스로 임베딩 테이블에서 행을 가져온다
print(lookup_result) # 1행: you 2행: need 3행:to 4행: run(embedding_table에 없어서 0으로 나옴)



{'know': 2, 'to': 3, 'you': 4, 'code': 5, 'need': 6, 'how': 7, '<unk>': 0, '<pad>': 1}
tensor([4, 6, 3, 0])
tensor([[0.2000, 0.1000, 0.8000],
        [0.1000, 0.8000, 0.9000],
        [0.1000, 0.5000, 0.7000],
        [0.0000, 0.0000, 0.0000]])


In [ ]:
# nn.Embedding() 사용하기

 # 전처리
train_data = 'you need to know how to code'
word_set = set(train_data.split()) # 중복을 제거한 단어들의 집합인 단어 집합 생성
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}  # 단어 집합의 각 단어에 고유한 정수 맵핑
vocab['<unk>'] = 0 
vocab['<pad>'] = 1


import torch.nn as nn
# 임베딩 테이블 만들기
embedding_layer = nn.Embedding(num_embeddings = len(vocab), # 임베딩을 할 단어들의 개수 == 단어 집합의 크기
                               embedding_dim = 3, # 임베딩 할 벡터의 차원. 사용자가 정해주는 하이퍼파라미터
                               padding_idx = 1) # 선택적으로 사용하는 인자, 패딩을 위한 토큰의 인덱스

print(embedding_layer.weight) # 단어 집합의 크기의 행을 가지는 임베딩 테이블

Parameter containing:
tensor([[ 1.1968, -0.6183,  0.2987],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.5569, -0.5075,  1.2371],
        [-1.9764, -0.0103,  1.9710],
        [ 1.9768,  1.5862, -0.4923],
        [ 1.4437,  0.7570, -1.0409],
        [-1.2703, -0.3534,  1.6235],
        [-1.4223, -0.0204, -0.6342]], requires_grad=True)


**9장 Chap8 사전 훈련된 워드 임베딩**

훈련 데이터가 적다면 파이토치의 nn.Embedding()으로 해당 문제에 충분히 특화된 임베딩 벡터를 만들어내는 것이 어려움 

=> 보다 일반적이고 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용할 수 있음

In [ ]:
# 훈련 데이터
from torchtext.legacy import data, datasets

# 필드 정의 
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

#  torchtext.datasets => IMDB 데이터셋 다운로드, 학습 데이터셋과 테스트 데이터셋으로 나누기
trainset, testset = datasets.IMDB.splits(TEXT, LABEL) # 훈련 데이터: 25,000개


# 1. 직접 훈련시킨 사전 훈련된 워드 임베딩을 사용하는 방법
 # 생략

# 2. 토치텍스트에서 제공하는 사전 훈련된 워드 임베딩을 사용하는 방법
from torchtext.vocab import GloVe
 # build_vocab을 통해 토치텍스트가 제공하는 사전 훈련된 임베딩 벡터를 다운
TEXT.build_vocab(trainset, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
  #  max_size: 단어 집합의 크기 제한 / min_freq: 최소 등장 빈도수
LABEL.build_vocab(trainset)

 # 확인해보기
print(TEXT.vocab.stoi) 
print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))
print(TEXT.vocab.vectors[0]) # <unk>의 임베딩 벡터 값 : 0으로 초기화된 상태
print(TEXT.vocab.vectors[1]) # <pad>의 임베딩 벡터 값 : 0으로 초기화된 상태
print(TEXT.vocab.vectors[10]) # this의 임베딩 벡터 값 : 이번에 다운로드 받은 단어 this의 사전 훈련된 임베딩 벡터 값
print(TEXT.vocab.vectors[9999]) # 'seeing'의 임베딩 벡터 값: 0으로 초기화된 상태 <= 사전훈련된 임베딩 벡터의 단어장에 'seeing' 존재 x

 # **TEXT.vocab.vectors를 nn.Embedding()의 초기화 입력으로 사용
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)
embedding_layer(torch.LongTensor([10])) # 단어 this의 임베딩 벡터값

tensor([[-2.0437e-01,  1.6431e-01,  4.1794e-02, -1.3708e-01, -2.9779e-01,
          3.3440e-01, -6.9955e-02, -6.8036e-02,  1.0604e-01, -2.0337e+00,
          1.7977e-01, -7.7403e-02, -1.9518e-01,  1.8324e-01,  3.0017e-02,
         -5.4762e-02, -4.5725e-01, -2.4509e-02,  5.7387e-02, -3.4878e-01,
          3.9696e-02,  4.4826e-01, -5.8462e-02,  4.1181e-01, -3.5411e-02,
         -1.4722e-01,  1.0740e-01, -2.5896e-01, -1.1658e-01,  1.9822e-01,
          3.2850e-01,  2.4177e-01, -5.7177e-01, -5.6442e-02, -9.6437e-01,
          3.4482e-01,  5.4639e-02,  2.3828e-01, -1.9139e-01,  3.0899e-01,
          2.8044e-01, -3.3814e-02, -2.5436e-01,  1.5373e-02,  1.6341e-01,
          2.6352e-01,  1.5812e-01,  3.2044e-01, -2.3082e-01,  2.6050e-01,
          2.0606e-01, -8.9769e-02, -1.0055e-01,  7.0378e-02, -2.7452e-02,
          2.7959e-01, -9.5862e-02,  2.0574e-01,  2.9522e-01, -1.2285e-02,
          1.1164e-01, -5.1373e-02,  4.6106e-01,  2.3014e-02, -3.7141e-01,
         -2.4166e-01,  3.3773e-02,  3.

QUIZ ) 모델에 대한 설명이 다음과 같을 때, 총 파라미터 개수를 구해보세요.

1. Embedding을 사용하며, 단어 집합(Vocabulary)의 크기가 5,000이고 임베딩 벡터의 차원은 100입니다.

2. 은닉층에서는 Simple RNN을 사용하며, 은닉 상태의 크기는 128입니다.

3. 훈련에 사용하는 모든 샘플의 길이는 30으로 가정합니다.

4. 이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용합니다.

5. 은닉층은 1개입니다.

1. Embedding = input(5000) x embedding (100) => 500000

2. Wx = 100(embedding) * 128(hidden) = 12,800

3. Wh = 128 * 128 = 16,384

4. bias(hidden)  = 128

5. Wy = 128
bias(output) = 1 

=> 총합 529,441